# Clasificación de Sentimientos Utilizando Naïve Bayes
Realizado por:

Paula Monge
Emmanuel Zúñiga
Trabajo de Invetigación del Curso:

Investigación en Ciencias de la Computación
Tema de Investigación:

Aprendizaje Automático en la clasificación de Sentimientos en Twitter: Comparación de los Algoritmos de LSTM Bidireccional, BERT y Naïve Bayes

# Importación de librerias necesarias 📚

In [1]:
import re
import seaborn as sns #for visualization
import matplotlib.pyplot as plt #for visualization
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive

## Carga del dataset 📅

In [2]:
# Se carga la libreria drive de google para cargar los archivos almacenados en drive
drive.mount("/content/gdrive")

# El dataset en cuestión contiene datos sobre tweets en diferentes idiomas, pero nos interesa utilizar solo
# los que se encuentran en español.
df = pd.read_csv('/content/gdrive/My Drive/imdb_new_dataset.csv')

mapping = {'positive': 1, 'negative': 0}
df['sentiment'] = df['sentiment'].map(mapping)

Mounted at /content/gdrive


Visualización de los primeros 10 registros del dataset

In [3]:
df.head(10)

,Unnamed: 0.1,Unnamed: 0,review_en,review_es,sentiment,clean_en,clean_es
0,0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,1,one reviewers mentioned watching 1 oz episode ...,criticos mencionado despues ver solo 1 oz epis...
1,1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,1,wonderful little production filming technique ...,pequena pequena produccion tecnica filmacion i...
2,2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,1,thought wonderful way spend time hot summer we...,pense manera maravillosa pasar tiempo fin sema...
3,3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",0,basically theres family little boy jake thinks...,basicamente familia nino pequeno jake piensa z...
4,4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",1,petter matteis love time money visually stunni...,amor tiempo petter mattei pelicula visualmente...
5,5,5,"Probably my all-time favorite movie, a story o...",Probablemente mi película favorita de todos lo...,1,probably time favorite movie story selflessnes...,probablemente pelicula favorita tiempos histor...
6,6,6,I sure would like to see a resurrection of a u...,Seguro que me gustaría ver una resurrección de...,1,sure would like see resurrection dated seahunt...,seguro gustaria ver resurreccion serie seahunt...
7,7,7,"This show was an amazing, fresh & innovative i...","Este espectáculo fue una idea increíble, fresc...",0,show amazing fresh innovative idea 70s first a...,espectaculo idea increible fresca innovadora a...
8,8,8,Encouraged by the positive comments about this...,Alentados por los comentarios positivos sobre ...,0,encouraged positive comments film looking forw...,alentados comentarios positivos pelicula aqui ...
9,9,9,If you like original gut wrenching laughter yo...,"Si te gusta la risa original desgarradora, te ...",1,like original gut wrenching laughter like movi...,si gusta risa original desgarradora gustara pe...


In [4]:
df.shape

(49602, 7)

## Preprocesamiento y limpieza de datos 🧹

In [5]:
df = df.dropna()

In [6]:
df.shape

(49602, 7)

## Creación de datos de entrenamiento y prueba 🏋

Se define una proporción de 80/20 para la creación de los datos de prueba y testeo.

In [7]:
train_pct = 0.8
train_size = round(len(df) * train_pct)

train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

test_data = test_data.drop('sentiment', axis=1)

In [8]:
train_data.head(20)

,Unnamed: 0.1,Unnamed: 0,review_en,review_es,sentiment,clean_en,clean_es
0,0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,1,one reviewers mentioned watching 1 oz episode ...,criticos mencionado despues ver solo 1 oz epis...
1,1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,1,wonderful little production filming technique ...,pequena pequena produccion tecnica filmacion i...
2,2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,1,thought wonderful way spend time hot summer we...,pense manera maravillosa pasar tiempo fin sema...
3,3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",0,basically theres family little boy jake thinks...,basicamente familia nino pequeno jake piensa z...
4,4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",1,petter matteis love time money visually stunni...,amor tiempo petter mattei pelicula visualmente...
5,5,5,"Probably my all-time favorite movie, a story o...",Probablemente mi película favorita de todos lo...,1,probably time favorite movie story selflessnes...,probablemente pelicula favorita tiempos histor...
6,6,6,I sure would like to see a resurrection of a u...,Seguro que me gustaría ver una resurrección de...,1,sure would like see resurrection dated seahunt...,seguro gustaria ver resurreccion serie seahunt...
7,7,7,"This show was an amazing, fresh & innovative i...","Este espectáculo fue una idea increíble, fresc...",0,show amazing fresh innovative idea 70s first a...,espectaculo idea increible fresca innovadora a...
8,8,8,Encouraged by the positive comments about this...,Alentados por los comentarios positivos sobre ...,0,encouraged positive comments film looking forw...,alentados comentarios positivos pelicula aqui ...
9,9,9,If you like original gut wrenching laughter yo...,"Si te gusta la risa original desgarradora, te ...",1,like original gut wrenching laughter like movi...,si gusta risa original desgarradora gustara pe...


In [50]:
test_data.head(10)

,Unnamed: 0.1,Unnamed: 0,review_en,review_es,clean_en,clean_es
39682,39932,39932,This is easily the worst movie i have EVER see...,Esta es fácilmente la peor película que he vis...,easily worst movie ever seen im exaggerating t...,facilmente peor pelicula visto exagerando dije...
39683,39934,39934,"""Admissions"" is a fine drama even though they'...","""Admisiones"" es un buen drama a pesar de que s...",admissions fine drama even though theyre probl...,admisiones buen drama pesar problemas final la...
39684,39935,39935,The movie started very well..so far Isabelle's...,"La película comenzó muy bien ... por ejemplo, ...",movie started well far isabelles exorcism coul...,pelicula comenzo bien ejemplo exorcismo isabel...
39685,39936,39936,"Now let me tell you about this movie, this mov...","¡Ahora déjame contarte sobre esta película, es...",let tell movie movie favorite movie movie exce...,ahora dejame contarte pelicula pelicula pelicu...
39686,39937,39937,it's a good watch if u have time - deals with ...,"Es un buen reloj si tienes tiempo, se ocupa de...",good watch u time deals three friends get need...,buen reloj si tiempo ocupa tres amigos ponen l...
39687,39938,39938,I never saw this movie until I bought the tape...,Nunca vi esta película hasta que compré la cin...,never saw movie bought tape last year enthrall...,nunca vi pelicula compre cinta ano pasado caut...
39688,39939,39939,"In 1904 Tangier, a wealthy American woman and ...","En 1904, Tánger, una mujer estadounidense adin...",1904 tangier wealthy american woman two childr...,1904 tanger mujer estadounidense adinerada dos...
39689,39940,39940,OK Hollywood is not liberal.Obviously I'm liei...,"Ok, Hollywood no es liberal. Observmente me es...",ok hollywood liberal obviously im lieing im co...,ok hollywood liberal observmente miente conser...
39690,39941,39941,This is the result of the town of Milpitas Cal...,Este es el resultado de la ciudad de Milpitas ...,result town milpitas california making home mo...,resultado ciudad milpitas california haciendo ...
39691,39942,39942,"Also known in a different form as ""House of Ex...","También conocido en una forma diferente como ""...",also known different form house exorcism messy...,tambien conocido forma diferente casa exorcism...


## Tokenización de los datos 🔡

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

In [52]:
sample_data = ["This is my first paper", "This is the second document", "here is the third thi one"]

In [53]:
vectorizer = CountVectorizer()

In [54]:
X = vectorizer.fit_transform(sample_data)

In [55]:
vectorizer.get_feature_names_out()

array(['document', 'first', 'here', 'is', 'my', 'one', 'paper', 'second',
       'the', 'thi', 'third', 'this'], dtype=object)

In [56]:
X.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0]])

In [57]:
data = train_data['clean_es'].tolist()

In [58]:
data_countvectorizer = CountVectorizer(dtype=np.uint8).fit_transform(data)

In [59]:
data_countvectorizer.shape

(39682, 156294)

In [60]:
X = data_countvectorizer
y = train_data['sentiment']

## Implementación de Naïve Bayes para clasificación 🤖

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
from sklearn.utils import shuffle
test_size = int(X.shape[0] * 0.2)


# Divide los datos en conjuntos de entrenamiento y prueba
X_test = X[:test_size]
X_train = X[test_size:]
y_test = y[:test_size]
y_train = y[test_size:]

In [63]:
from sklearn.naive_bayes import MultinomialNB #for discrete values

In [64]:
alpha = 1
fit_prior = False

NB_classifier = MultinomialNB(alpha=alpha, fit_prior=fit_prior)

In [65]:
NB_classifier.fit(X_train, y_train)

MultinomialNB(alpha=1, fit_prior=False)

In [66]:
y_pred = NB_classifier.predict(X_test)

In [67]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [68]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[3413,  547],
       [ 643, 3333]])

In [69]:
ac = accuracy_score(y_test, y_pred)
ac

0.8500504032258065

In [70]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      3960
           1       0.86      0.84      0.85      3976

    accuracy                           0.85      7936
   macro avg       0.85      0.85      0.85      7936
weighted avg       0.85      0.85      0.85      7936

